# Quantum Algorithms Framework Demo

This notebook demonstrates the quantum algorithm base classes and framework in quactuary:
- QuantumAlgorithm abstract base class and 4-step workflow
- Implementing custom quantum algorithms
- Variational quantum algorithms
- Monte Carlo quantum algorithms
- Integration with classical fallbacks

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Any, List, Optional
from abc import ABC, abstractmethod

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.primitives import Estimator, Sampler
from qiskit_algorithms.optimizers import COBYLA, SPSA

# Import quactuary quantum modules
from quactuary.quantum.base_quantum import (
    QuantumAlgorithm,
    StatePreparationAlgorithm,
    VariationalQuantumAlgorithm
)
from quactuary.quantum.algorithms.base_algorithm import (
    ActuarialQuantumAlgorithm,
    ProbabilityDistributionLoader,
    MonteCarloQuantumAlgorithm
)
from quactuary.quantum.quantum_types import (
    CircuitMetrics,
    OptimizationResult,
    QuantumError
)
from quactuary.quantum.circuits.builders import CircuitBuilder, ParameterizedCircuitBuilder
from quactuary.quantum.circuits.templates import create_hardware_efficient_ansatz

## 1. Understanding the 4-Step Quantum Algorithm Workflow

The QuantumAlgorithm base class enforces a structured 4-step workflow following Qiskit best practices.

In [ ]:
# Example: Simple quantum algorithm implementation
class SimpleQuantumAlgorithm(QuantumAlgorithm):
    """Simple example demonstrating the 4-step workflow."""
    
    def __init__(self, num_qubits: int = 3):
        super().__init__()
        self.num_qubits = num_qubits
    
    @property
    def required_qubits(self) -> int:
        return self.num_qubits
    
    def build_circuit(self, **params) -> QuantumCircuit:
        """Step 1: Map problem to quantum-native format."""
        print("Step 1: Building quantum circuit...")
        
        # Get rotation angle from params
        angle = params.get('angle', np.pi/4)
        
        # Build circuit
        qc = QuantumCircuit(self.num_qubits)
        
        # Create superposition
        qc.h(range(self.num_qubits))
        
        # Apply rotations
        for i in range(self.num_qubits):
            qc.ry(angle * (i + 1), i)
        
        # Entangle
        for i in range(self.num_qubits - 1):
            qc.cx(i, i + 1)
        
        return qc
    
    def optimize_circuit(self, circuit: QuantumCircuit) -> QuantumCircuit:
        """Step 2: Optimize circuits using transpile."""
        print("Step 2: Optimizing circuit...")
        
        # For demo, just return the circuit
        # In practice, would use transpile with backend
        return circuit
    
    def execute(self, circuit: QuantumCircuit) -> Dict[str, Any]:
        """Step 3: Execute using quantum primitives."""
        print("Step 3: Executing circuit...")
        
        # Use statevector simulation
        statevector = Statevector.from_instruction(circuit)
        
        return {
            'statevector': statevector,
            'probabilities': statevector.probabilities()
        }
    
    def analyze_results(self, results: Dict[str, Any]) -> Any:
        """Step 4: Analyze quantum results."""
        print("Step 4: Analyzing results...")
        
        probs = results['probabilities']
        
        # Find most likely outcome
        max_idx = np.argmax(probs)
        max_prob = probs[max_idx]
        
        return {
            'most_likely_state': format(max_idx, f'0{self.num_qubits}b'),
            'probability': max_prob,
            'entropy': -np.sum(probs * np.log2(probs + 1e-10))
        }
    
    def classical_equivalent(self, **params) -> Any:
        """Classical simulation for comparison."""
        # Simple classical approximation
        angle = params.get('angle', np.pi/4)
        
        # Simulate expected outcome classically
        prob_zero = np.cos(angle/2)**2
        
        return {
            'most_likely_state': '0' * self.num_qubits,
            'probability': prob_zero**self.num_qubits,
            'entropy': self.num_qubits  # Maximum entropy approximation
        }

# Demonstrate the algorithm
algo = SimpleQuantumAlgorithm(num_qubits=3)

# Run the complete workflow
print("Running quantum algorithm workflow:")
print("=" * 40)
quantum_result = algo.run(angle=np.pi/3)

print("\nQuantum Results:")
for key, value in quantum_result.items():
    print(f"  {key}: {value}")

# Compare with classical
classical_result = algo.classical_equivalent(angle=np.pi/3)
print("\nClassical Approximation:")
for key, value in classical_result.items():
    print(f"  {key}: {value}")

# Get circuit info
print("\nCircuit Information:")
info = algo.get_circuit_info()
for key, value in info.items():
    print(f"  {key}: {value}")

In [ ]:
# Visualize the circuit
circuit = algo.build_circuit(angle=np.pi/3)
circuit.draw(output='mpl', style='iqp')

## 2. ActuarialQuantumAlgorithm: Enhanced Base Class

The ActuarialQuantumAlgorithm extends the base class with actuarial-specific features.

In [ ]:
# Example: Risk measure calculation algorithm
class QuantumRiskMeasureAlgorithm(ActuarialQuantumAlgorithm):
    """Quantum algorithm for calculating risk measures."""
    
    def __init__(self, loss_distribution: np.ndarray, confidence_level: float = 0.95):
        super().__init__()
        self.loss_distribution = loss_distribution / np.sum(loss_distribution)
        self.confidence_level = confidence_level
        self._loss_values = np.arange(len(loss_distribution))
    
    @property
    def required_qubits(self) -> int:
        return int(np.ceil(np.log2(len(self.loss_distribution))))
    
    def validate_input(self, **params) -> None:
        """Validate algorithm inputs."""
        if np.any(self.loss_distribution < 0):
            raise ValueError("Loss distribution cannot have negative values")
        
        if not 0 < self.confidence_level < 1:
            raise ValueError("Confidence level must be between 0 and 1")
    
    def build_circuit(self, **params) -> QuantumCircuit:
        """Build circuit for risk measure calculation."""
        # Load the loss distribution
        loader = ProbabilityDistributionLoader(self.loss_distribution)
        circuit = loader.build_circuit()
        
        # Add measurement
        circuit.measure_all()
        
        return circuit
    
    def execute(self, circuit: QuantumCircuit) -> Dict[str, Any]:
        """Execute with sampling for VaR estimation."""
        # Use sampler for measurement outcomes
        job = self.sampler.run(circuit, shots=10000)
        result = job.result()
        
        # Convert quasi-probabilities to counts
        quasi_probs = result.quasi_dists[0]
        
        return {
            'quasi_probabilities': quasi_probs,
            'metadata': result.metadata
        }
    
    def analyze_results(self, results: Dict[str, Any]) -> Dict[str, float]:
        """Calculate VaR and CVaR from quantum results."""
        quasi_probs = results['quasi_probabilities']
        
        # Convert to probabilities array
        probs = np.zeros(len(self.loss_distribution))
        for outcome, prob in quasi_probs.items():
            if outcome < len(probs):
                probs[outcome] = prob
        
        # Normalize
        probs = probs / np.sum(probs)
        
        # Calculate VaR
        cumsum = np.cumsum(probs)
        var_idx = np.argmax(cumsum >= self.confidence_level)
        var = self._loss_values[var_idx]
        
        # Calculate CVaR (Conditional VaR)
        tail_probs = probs[var_idx:] / np.sum(probs[var_idx:])
        cvar = np.sum(self._loss_values[var_idx:] * tail_probs)
        
        # Expected loss
        expected_loss = np.sum(self._loss_values * probs)
        
        return {
            'VaR': var,
            'CVaR': cvar,
            'expected_loss': expected_loss,
            'confidence_level': self.confidence_level
        }
    
    def classical_equivalent(self, **params) -> Dict[str, float]:
        """Classical calculation of risk measures."""
        # Calculate VaR classically
        cumsum = np.cumsum(self.loss_distribution)
        var_idx = np.argmax(cumsum >= self.confidence_level)
        var = self._loss_values[var_idx]
        
        # Calculate CVaR
        tail_probs = self.loss_distribution[var_idx:] / np.sum(self.loss_distribution[var_idx:])
        cvar = np.sum(self._loss_values[var_idx:] * tail_probs)
        
        # Expected loss
        expected_loss = np.sum(self._loss_values * self.loss_distribution)
        
        return {
            'VaR': var,
            'CVaR': cvar,
            'expected_loss': expected_loss,
            'confidence_level': self.confidence_level
        }
    
    def get_resource_requirements(self, **params) -> Dict[str, Any]:
        """Estimate resource requirements."""
        base_reqs = super().get_resource_requirements(**params)
        
        # Add specific estimates
        base_reqs.update({
            'shots_recommended': 10000,
            'memory_mb': len(self.loss_distribution) * 8 / 1e6,
            'classical_comparison_available': True
        })
        
        return base_reqs

# Example usage
# Create a sample loss distribution
np.random.seed(42)
losses = np.random.gamma(2, 2, 1000)
hist, bin_edges = np.histogram(losses, bins=16)
loss_probs = hist / np.sum(hist)
loss_values = (bin_edges[:-1] + bin_edges[1:]) / 2

# Create and run the algorithm
risk_algo = QuantumRiskMeasureAlgorithm(loss_probs, confidence_level=0.95)

# Validate inputs
risk_algo.validate_input()

# Get resource requirements
print("Resource Requirements:")
reqs = risk_algo.get_resource_requirements()
for key, value in reqs.items():
    print(f"  {key}: {value}")

# Run classical calculation
classical_risk = risk_algo.classical_equivalent()
print("\nClassical Risk Measures:")
for key, value in classical_risk.items():
    if isinstance(value, float) and key != 'confidence_level':
        print(f"  {key}: ${value*1000:.2f}")
    else:
        print(f"  {key}: {value}")

## 3. Variational Quantum Algorithms

Implementing variational algorithms like VQE for optimization problems.

In [ ]:
# Example: Portfolio optimization using variational algorithm
class QuantumPortfolioOptimizer(VariationalQuantumAlgorithm):
    """Quantum portfolio optimization using variational approach."""
    
    def __init__(self, returns: np.ndarray, cov_matrix: np.ndarray, 
                 risk_aversion: float = 1.0):
        super().__init__(optimizer=COBYLA(maxiter=100))
        self.returns = returns
        self.cov_matrix = cov_matrix
        self.risk_aversion = risk_aversion
        self.n_assets = len(returns)
    
    @property
    def required_qubits(self) -> int:
        return self.n_assets
    
    def build_circuit(self, **params) -> QuantumCircuit:
        """Build parameterized circuit for portfolio optimization."""
        # Use hardware-efficient ansatz
        circuit = create_hardware_efficient_ansatz(
            num_qubits=self.n_assets, 
            depth=3
        )
        return circuit
    
    def cost_function(self, params: np.ndarray) -> float:
        """Evaluate portfolio cost function."""
        # Bind parameters to circuit
        circuit = self._circuit.assign_parameters(params)
        
        # Get statevector
        statevector = Statevector.from_instruction(circuit)
        probs = statevector.probabilities()
        
        # Map probabilities to portfolio weights
        weights = self._probs_to_weights(probs)
        
        # Calculate portfolio metrics
        expected_return = np.dot(weights, self.returns)
        portfolio_risk = np.sqrt(weights @ self.cov_matrix @ weights)
        
        # Cost function: maximize return - risk_aversion * risk
        cost = -(expected_return - self.risk_aversion * portfolio_risk)
        
        return cost
    
    def _probs_to_weights(self, probs: np.ndarray) -> np.ndarray:
        """Convert quantum probabilities to portfolio weights."""
        # Simple mapping: use first 2^n probabilities as weights
        weights = probs[:self.n_assets]
        # Normalize
        return weights / np.sum(weights)
    
    def get_initial_params(self) -> np.ndarray:
        """Generate initial parameters."""
        # Random initial parameters
        return np.random.uniform(0, 2*np.pi, self._circuit.num_parameters)
    
    def analyze_results(self, results: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze optimization results."""
        optimal_params = self._optimal_params
        
        # Get optimal circuit
        optimal_circuit = self._circuit.assign_parameters(optimal_params)
        statevector = Statevector.from_instruction(optimal_circuit)
        probs = statevector.probabilities()
        
        # Get optimal weights
        weights = self._probs_to_weights(probs)
        
        # Calculate portfolio metrics
        expected_return = np.dot(weights, self.returns)
        portfolio_risk = np.sqrt(weights @ self.cov_matrix @ weights)
        sharpe_ratio = expected_return / portfolio_risk
        
        return {
            'weights': weights,
            'expected_return': expected_return,
            'risk': portfolio_risk,
            'sharpe_ratio': sharpe_ratio
        }
    
    def classical_equivalent(self, **params) -> Dict[str, Any]:
        """Classical mean-variance optimization."""
        # Use equal weights as simple classical baseline
        weights = np.ones(self.n_assets) / self.n_assets
        
        expected_return = np.dot(weights, self.returns)
        portfolio_risk = np.sqrt(weights @ self.cov_matrix @ weights)
        sharpe_ratio = expected_return / portfolio_risk
        
        return {
            'weights': weights,
            'expected_return': expected_return,
            'risk': portfolio_risk,
            'sharpe_ratio': sharpe_ratio
        }

# Example portfolio optimization
# Create sample data
n_assets = 4
np.random.seed(42)

# Expected returns (annualized)
returns = np.array([0.10, 0.12, 0.15, 0.08])

# Covariance matrix
volatilities = np.array([0.15, 0.20, 0.25, 0.12])
correlation = np.array([
    [1.0, 0.3, 0.2, 0.1],
    [0.3, 1.0, 0.4, 0.2],
    [0.2, 0.4, 1.0, 0.3],
    [0.1, 0.2, 0.3, 1.0]
])
cov_matrix = np.outer(volatilities, volatilities) * correlation

# Create optimizer
portfolio_opt = QuantumPortfolioOptimizer(returns, cov_matrix, risk_aversion=2.0)

# Build circuit
portfolio_opt._circuit = portfolio_opt.build_circuit()
print(f"Circuit parameters: {portfolio_opt._circuit.num_parameters}")
print(f"Circuit depth: {portfolio_opt._circuit.depth()}")

# Classical baseline
classical_portfolio = portfolio_opt.classical_equivalent()
print("\nClassical Portfolio (Equal Weights):")
print(f"  Weights: {classical_portfolio['weights']}")
print(f"  Expected Return: {classical_portfolio['expected_return']:.2%}")
print(f"  Risk: {classical_portfolio['risk']:.2%}")
print(f"  Sharpe Ratio: {classical_portfolio['sharpe_ratio']:.3f}")

## 4. Monte Carlo Quantum Algorithms

Quantum algorithms for Monte Carlo sampling and estimation.

In [ ]:
# Example: Quantum Monte Carlo for option pricing
class QuantumOptionPricer(MonteCarloQuantumAlgorithm):
    """Quantum Monte Carlo for European option pricing."""
    
    def __init__(self, S0: float, K: float, r: float, sigma: float, 
                 T: float, n_qubits: int = 4):
        super().__init__(num_samples=1000)
        self.S0 = S0  # Initial stock price
        self.K = K    # Strike price
        self.r = r    # Risk-free rate
        self.sigma = sigma  # Volatility
        self.T = T    # Time to maturity
        self.n_qubits = n_qubits
        self.n_paths = 2**n_qubits
    
    @property
    def required_qubits(self) -> int:
        return self.n_qubits + 1  # +1 for ancilla
    
    def build_circuit(self, **params) -> QuantumCircuit:
        """Build circuit for option pricing."""
        qr = QuantumRegister(self.n_qubits, 'path')
        ar = QuantumRegister(1, 'ancilla')
        cr = ClassicalRegister(self.n_qubits + 1, 'measure')
        
        qc = QuantumCircuit(qr, ar, cr)
        
        # Create superposition of paths
        qc.h(qr)
        
        # Encode price evolution (simplified)
        for i in range(self.n_qubits):
            angle = self.sigma * np.sqrt(self.T) * (i + 1) / self.n_qubits
            qc.ry(angle, qr[i])
        
        # Mark in-the-money paths (simplified)
        # In practice, would use more sophisticated comparison
        qc.x(ar[0])  # Start with |1>
        for i in range(self.n_qubits):
            qc.cx(qr[i], ar[0])
        
        # Measure all qubits
        qc.measure(qr, cr[:self.n_qubits])
        qc.measure(ar, cr[self.n_qubits])
        
        return qc
    
    def analyze_results(self, results: Dict[str, Any]) -> Dict[str, float]:
        """Analyze Monte Carlo results for option price."""
        counts = results['counts']
        
        # Count in-the-money paths
        itm_count = 0
        total_count = 0
        
        for outcome, count in counts.items():
            total_count += count
            # Check ancilla bit (last bit)
            if outcome >> self.n_qubits == 1:
                itm_count += count
        
        # Estimate probability
        itm_prob = itm_count / total_count if total_count > 0 else 0
        
        # Simple option price estimate
        option_price = self.S0 * itm_prob * np.exp(-self.r * self.T)
        
        # Error estimate
        error = np.sqrt(itm_prob * (1 - itm_prob) / total_count) * self.S0
        
        return {
            'option_price': option_price,
            'error_estimate': error,
            'itm_probability': itm_prob,
            'total_samples': total_count
        }
    
    def classical_equivalent(self, **params) -> Dict[str, float]:
        """Black-Scholes formula for comparison."""
        from scipy.stats import norm
        
        d1 = (np.log(self.S0/self.K) + (self.r + 0.5*self.sigma**2)*self.T) / (self.sigma*np.sqrt(self.T))
        d2 = d1 - self.sigma*np.sqrt(self.T)
        
        call_price = self.S0*norm.cdf(d1) - self.K*np.exp(-self.r*self.T)*norm.cdf(d2)
        
        return {
            'option_price': call_price,
            'error_estimate': 0,  # Analytical solution has no error
            'itm_probability': norm.cdf(d2),
            'total_samples': 0  # Not applicable
        }

# Example option pricing
option_params = {
    'S0': 100,      # Current stock price
    'K': 105,       # Strike price
    'r': 0.05,      # Risk-free rate
    'sigma': 0.2,   # Volatility
    'T': 0.25,      # 3 months to maturity
    'n_qubits': 5   # Number of qubits for paths
}

option_pricer = QuantumOptionPricer(**option_params)

# Build and visualize circuit
option_circuit = option_pricer.build_circuit()
print(f"Option pricing circuit with {option_pricer.required_qubits} qubits")
print(f"Simulating {option_pricer.n_paths} price paths")

# Classical Black-Scholes price
bs_price = option_pricer.classical_equivalent()
print("\nBlack-Scholes Option Price:")
for key, value in bs_price.items():
    if key == 'option_price':
        print(f"  {key}: ${value:.2f}")
    elif key == 'itm_probability':
        print(f"  {key}: {value:.2%}")
    else:
        print(f"  {key}: {value}")

## 5. Error Handling and Validation

Demonstrating error handling and validation in quantum algorithms.

In [ ]:
# Example: Robust quantum algorithm with error handling
class RobustQuantumAlgorithm(ActuarialQuantumAlgorithm):
    """Example showing comprehensive error handling."""
    
    def __init__(self, data: np.ndarray, max_qubits: int = 20):
        super().__init__()
        self.data = data
        self.max_qubits = max_qubits
    
    @property
    def required_qubits(self) -> int:
        return int(np.ceil(np.log2(len(self.data))))
    
    def validate_input(self, **params) -> None:
        """Comprehensive input validation."""
        # Check data size
        if len(self.data) == 0:
            raise ValueError("Data cannot be empty")
        
        # Check qubit requirements
        if self.required_qubits > self.max_qubits:
            raise QuantumError(
                f"Data size requires {self.required_qubits} qubits, "
                f"but maximum is {self.max_qubits}"
            )
        
        # Check for NaN or infinite values
        if np.any(np.isnan(self.data)) or np.any(np.isinf(self.data)):
            raise ValueError("Data contains NaN or infinite values")
        
        # Check optional parameters
        threshold = params.get('threshold', 0)
        if not isinstance(threshold, (int, float)):
            raise TypeError(f"Threshold must be numeric, got {type(threshold)}")
    
    def build_circuit(self, **params) -> QuantumCircuit:
        """Build circuit with error handling."""
        try:
            # Validate before building
            self.validate_input(**params)
            
            # Build circuit
            qc = QuantumCircuit(self.required_qubits)
            
            # Add gates based on data
            for i in range(min(len(self.data), 2**self.required_qubits)):
                if self.data[i] > params.get('threshold', 0):
                    qc.x(i % self.required_qubits)
            
            return qc
            
        except Exception as e:
            # Log error and re-raise with context
            raise QuantumError(f"Circuit construction failed: {str(e)}") from e
    
    def execute(self, circuit: QuantumCircuit) -> Dict[str, Any]:
        """Execute with fallback on error."""
        try:
            # Try quantum execution
            return super().execute(circuit)
            
        except Exception as e:
            # Fallback to classical
            print(f"Quantum execution failed: {e}")
            print("Falling back to classical simulation...")
            
            return {
                'fallback': True,
                'classical_result': self.classical_equivalent()
            }
    
    def analyze_results(self, results: Dict[str, Any]) -> Any:
        """Analyze with fallback handling."""
        if results.get('fallback', False):
            return results['classical_result']
        
        # Normal analysis
        return {'data_mean': np.mean(self.data)}
    
    def classical_equivalent(self, **params) -> Any:
        """Classical computation."""
        return {'data_mean': np.mean(self.data)}

# Test error handling
print("Testing error handling scenarios:\n")

# Scenario 1: Valid input
try:
    algo1 = RobustQuantumAlgorithm(np.array([1, 2, 3, 4]))
    algo1.validate_input(threshold=2.5)
    print("✓ Scenario 1: Valid input passed")
except Exception as e:
    print(f"✗ Scenario 1 failed: {e}")

# Scenario 2: Too much data
try:
    algo2 = RobustQuantumAlgorithm(np.ones(2**21), max_qubits=20)
    algo2.validate_input()
    print("✗ Scenario 2: Should have failed")
except QuantumError as e:
    print(f"✓ Scenario 2: Correctly caught - {e}")

# Scenario 3: Invalid data
try:
    algo3 = RobustQuantumAlgorithm(np.array([1, 2, np.nan, 4]))
    algo3.validate_input()
    print("✗ Scenario 3: Should have failed")
except ValueError as e:
    print(f"✓ Scenario 3: Correctly caught - {e}")

# Scenario 4: Invalid parameter type
try:
    algo4 = RobustQuantumAlgorithm(np.array([1, 2, 3, 4]))
    algo4.validate_input(threshold="invalid")
    print("✗ Scenario 4: Should have failed")
except TypeError as e:
    print(f"✓ Scenario 4: Correctly caught - {e}")

## 6. Algorithm Comparison and Benchmarking

Compare quantum and classical algorithm performance.

In [ ]:
# Benchmark different algorithm implementations
import time

def benchmark_algorithm(algorithm: QuantumAlgorithm, name: str, **params):
    """Benchmark quantum algorithm performance."""
    results = {
        'name': name,
        'qubits': algorithm.required_qubits
    }
    
    # Time circuit construction
    start = time.time()
    circuit = algorithm.build_circuit(**params)
    results['build_time'] = time.time() - start
    
    # Get circuit metrics
    metrics = CircuitMetrics.from_circuit(circuit)
    results['depth'] = metrics.depth
    results['gates'] = metrics.gate_count
    results['cnots'] = metrics.cnot_count
    
    # Time classical equivalent
    start = time.time()
    classical_result = algorithm.classical_equivalent(**params)
    results['classical_time'] = time.time() - start
    
    return results

# Create test algorithms
algorithms = [
    (SimpleQuantumAlgorithm(3), "Simple Algorithm", {'angle': np.pi/4}),
    (ProbabilityDistributionLoader([0.1, 0.2, 0.3, 0.4]), "Distribution Loader", {}),
    (QuantumRiskMeasureAlgorithm(loss_probs[:8]), "Risk Measure (8 states)", {}),
    (QuantumOptionPricer(**option_params), "Option Pricer", {})
]

# Run benchmarks
benchmark_results = []
for algo, name, params in algorithms:
    try:
        result = benchmark_algorithm(algo, name, **params)
        benchmark_results.append(result)
    except Exception as e:
        print(f"Failed to benchmark {name}: {e}")

# Display results
print("Algorithm Benchmarking Results")
print("=" * 100)
print(f"{'Algorithm':<25} {'Qubits':<8} {'Depth':<8} {'Gates':<8} {'CNOTs':<8} "
      f"{'Build (ms)':<12} {'Classical (ms)':<15}")
print("-" * 100)

for r in benchmark_results:
    print(f"{r['name']:<25} {r['qubits']:<8} {r['depth']:<8} {r['gates']:<8} "
          f"{r['cnots']:<8} {r['build_time']*1000:<12.2f} {r['classical_time']*1000:<15.2f}")

# Visualize metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

names = [r['name'] for r in benchmark_results]
depths = [r['depth'] for r in benchmark_results]
gates = [r['gates'] for r in benchmark_results]

# Circuit complexity
x = np.arange(len(names))
width = 0.35

ax1.bar(x - width/2, depths, width, label='Depth', color='skyblue')
ax1.bar(x + width/2, gates, width, label='Gate Count', color='lightcoral')
ax1.set_ylabel('Count')
ax1.set_title('Circuit Complexity')
ax1.set_xticks(x)
ax1.set_xticklabels(names, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Execution time
build_times = [r['build_time']*1000 for r in benchmark_results]
classical_times = [r['classical_time']*1000 for r in benchmark_results]

ax2.bar(x - width/2, build_times, width, label='Circuit Build', color='green')
ax2.bar(x + width/2, classical_times, width, label='Classical', color='orange')
ax2.set_ylabel('Time (ms)')
ax2.set_title('Execution Time')
ax2.set_xticks(x)
ax2.set_xticklabels(names, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated the quantum algorithms framework in quactuary:

1. **QuantumAlgorithm Base Class**: 4-step workflow (build, optimize, execute, analyze)
2. **ActuarialQuantumAlgorithm**: Enhanced base class with validation and resource estimation
3. **Variational Algorithms**: Portfolio optimization using parameterized circuits
4. **Monte Carlo Algorithms**: Option pricing with quantum sampling
5. **Error Handling**: Robust algorithm design with validation and fallbacks
6. **Benchmarking**: Performance comparison framework

Key design principles:
- Clear separation between quantum and classical implementations
- Mandatory classical fallback for all quantum algorithms
- Comprehensive error handling and validation
- Resource estimation and performance tracking
- Integration with Qiskit primitives (Estimator, Sampler)

The framework provides a solid foundation for implementing quantum algorithms for actuarial applications while maintaining compatibility with classical methods.